In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import pickle

def train_credit_model():
    # Load data
    df = pd.read_csv('data.csv')
    
    # Encode categoricals
    le = LabelEncoder()
    df['loan_type_encoded'] = le.fit_transform(df['loan_type'])
    df['new_repeat_encoded'] = le.fit_transform(df['New_versus_Repeat'])
    
    # Create features
    features = [
        'Total_Amount', 
        'duration',
        'loan_type_encoded',
        'new_repeat_encoded',
        'Lender_portion_Funded',
        'Amount_Funded_By_Lender'
    ]
    
    X = df[features]
    y = df['target']
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create pipeline
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('smote', SMOTE(random_state=42)),
        ('rf', RandomForestClassifier(
            n_estimators=200,
            max_depth=20,
            class_weight='balanced',
            random_state=42
        ))
    ])
    
    # Train
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    print("Model Performance:")
    print(classification_report(y_test, y_pred))
    
    # Save model
    with open('credit_scoring.pkl', 'wb') as f:
        pickle.dump(model, f)

    return model

if __name__ == "__main__":
    model = train_credit_model()

Model Performance:
              precision    recall  f1-score   support

           0       0.95      0.78      0.86        23
           1       0.17      0.50      0.25         2

    accuracy                           0.76        25
   macro avg       0.56      0.64      0.55        25
weighted avg       0.88      0.76      0.81        25

